#### Includes

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import altair as alt
from altair import datum
import pandas as pd
import time

In [3]:
import sources
from steered_cnn.utils import AttributeDict
from src.config import parse_config
from steered_cnn.models import *
from steered_cnn.steered_conv import SteeredConvBN, SteerableKernelBase
from steered_cnn.utils import ConvBN

## Init

In [4]:
cfg = """
experiment:
    name: 'test'
    sub-experiment: 'test'

model:
    steered: 
       steering: 'angle'
       base: {'kr': 5, 'max_k': 5}
    depth: 2
    nscale: 5
    nfeatures: 11
    padding: 'auto'
    upsampling: 'bilinear'
    downsampling: 'conv'
    batchnorm: True

hyper-parameters:
    batch-size: 4
"""


cfg = parse_config(cfg)

In [5]:
train, val, test = sources.load_dataset(cfg)

In [6]:
net = setup_model(cfg.model, 6, 1).cuda()
net

SteeredUNet(
  (downconv0-0): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downconv0-1): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downsample0): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(11, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downconv1-0): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): ReLU()
    )
  )
  (downconv1-1): SteeredConvBN(
    (conv): SteeredConv2d()
    (bn_relu): Sequential(
      (0): BatchNorm2d(22, eps=1e-05, momentum=0.1, affine=True, track_runni

## Run Model

In [7]:
for data in train:
    break

### Monitor Shapes

In [8]:
def hook(name):
    def print_shape(mod, inputs, outputs):
        print(name, ': ', [_.shape for _ in inputs], ' -> ' , outputs.shape)
    return print_shape
for name, mod in net.named_modules():
    if isinstance(mod, (SteeredConvBN, ConvBN)):
        mod.register_forward_hook(hook(name))

In [9]:
args = {'x': data['x'].cuda()}
if 'alpha' in data:
    args['alpha'] = data['alpha'].cuda()

r = net(**args)
print(data['x'].shape, ' -> ' ,r.shape)

downconv0-0 :  [torch.Size([4, 6, 565, 565])]  ->  torch.Size([4, 11, 565, 565])
downconv0-1 :  [torch.Size([4, 11, 565, 565])]  ->  torch.Size([4, 11, 565, 565])
downsample0 :  [torch.Size([4, 11, 565, 565])]  ->  torch.Size([4, 11, 283, 283])
downconv1-0 :  [torch.Size([4, 11, 283, 283])]  ->  torch.Size([4, 22, 283, 283])
downconv1-1 :  [torch.Size([4, 22, 283, 283])]  ->  torch.Size([4, 22, 283, 283])
downsample1 :  [torch.Size([4, 22, 283, 283])]  ->  torch.Size([4, 22, 142, 142])
downconv2-0 :  [torch.Size([4, 22, 142, 142])]  ->  torch.Size([4, 44, 142, 142])
downconv2-1 :  [torch.Size([4, 44, 142, 142])]  ->  torch.Size([4, 44, 142, 142])
downsample2 :  [torch.Size([4, 44, 142, 142])]  ->  torch.Size([4, 44, 72, 72])
downconv3-0 :  [torch.Size([4, 44, 72, 72])]  ->  torch.Size([4, 88, 72, 72])
downconv3-1 :  [torch.Size([4, 88, 72, 72])]  ->  torch.Size([4, 88, 72, 72])
downsample3 :  [torch.Size([4, 88, 72, 72])]  ->  torch.Size([4, 88, 37, 37])
downconv4-0 :  [torch.Size([4, 

/home/gaby/.conda/envs/nnet/lib/python3.7/site-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


In [11]:
kr = SteerableKernelBase.create_radial(5, cap_k=False, max_k=8).kernels_info
KR = {}
for _ in kr:
    k = _['k']
    r = _['r']
    if k not in KR:
        KR[k] = set()
    KR[k].add(r)
KR

{0: {0.0, 1.2071068, 2.325141, 3.3071315},
 1: {1.2071068, 2.325141, 3.3071315},
 2: {1.2071068, 2.325141, 3.3071315},
 3: {1.2071068, 2.325141, 3.3071315},
 4: {1.2071068, 2.325141, 3.3071315},
 5: {2.325141, 3.3071315},
 6: {2.325141, 3.3071315},
 7: {2.325141, 3.3071315},
 8: {2.325141, 3.3071315}}